In [ ]:
# 20250816

In [ ]:
# https://csp.gitbook.io/langchain-for-beginners/ch13-langchain-expression-language-lcel/08.-runnablewithmessagehistory

In [8]:
# 08. RunnableWithMessageHistory

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

In [10]:
# Load environment variables from .env file
load_dotenv()

True

In [13]:
model = ChatGroq(model="llama3-70b-8192")
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an assistant who is proficient in {ability}. Please respond in 20 characters or less.",
        ),
        # Use conversation history as a variable, history becomes the key of MessageHistory
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),  # Use user input as a variable
    ]
)
runnable = prompt | model  # Create a runnable object by connecting a prompt and a model

In [22]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_core.runnables.history import RunnableWithMessageHistory

In [23]:
store = {}  # A dictionary to store session records.

In [24]:
# Function to retrieve session records based on session ID
def get_session_history(session_ids: str) -> BaseChatMessageHistory:
    print(session_ids)
    if session_ids not in store:  # If the session ID is not in the store
        # Create a new ChatMessageHistory object and save it to the store.
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # Returns the session record for the given session ID.


In [25]:
with_message_history = (
    RunnableWithMessageHistory(  # RunnableWithMessageHistory Object creation
        runnable,  # Runnable object to execute
        get_session_history,  # Function to retrieve session records
        input_messages_key="input",  #Specifies the key to be processed with the latest input message
        history_messages_key="history",#Specifies the key to add the previous message.  
    )
)

In [26]:
try:
    if history_factory_config:
        _config_specs = history_factory_config
    else:
        # If not provided, then we'll use the default session_id field
        _config_specs = [
            ConfigurableFieldSpec(
                id="session_id",
                annotation=str,
                name="Session ID",
                description="Unique identifier for a session.",
                default="",
                is_shared=True,
            ),
        ]
except NameError:
    # Fallback if history_factory_config isn't defined at all
    _config_specs = [
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for a session.",
            default="",
            is_shared=True,
        ),
    ]


In [27]:
with_message_history.invoke(
    # Pass the math related question "What is the meaning of cosine?" as input.
    {"ability": "math", "input": "What does cosine mean?"},
    # Pass the session ID "abc123" as configuration information.
    config={"configurable": {"session_id": "abc123"}},
)

abc123


AIMessage(content='adjacent side / hypotenuse', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 39, 'total_tokens': 47, 'completion_time': 0.02764438, 'prompt_time': 0.012345383, 'queue_time': 0.143315083, 'total_time': 0.039989763}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_bf16903a67', 'finish_reason': 'stop', 'logprobs': None}, id='run--202e0022-2f9d-457c-8175-dad34cafa22c-0', usage_metadata={'input_tokens': 39, 'output_tokens': 8, 'total_tokens': 47})